<a href="https://colab.research.google.com/github/Durgakesav/Book-Recommendation-System/blob/main/ML_Lab_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from math import sqrt
import warnings

print("***************BOOK RECOMMENDATION SYSTEM********************")
print("**************************************************************")

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Load data
books = pd.read_excel('books_dataset.xlsx')
ratings = pd.read_excel('ratings.xlsx')
users = pd.read_excel('users.xlsx')

# Merge ratings with books
book_data = pd.merge(ratings, books, on='book_id', how='inner')

# Merge with user data
full_data = pd.merge(book_data, users, on='user_id', how='inner')

# One-hot encode categories
category_encoded = pd.get_dummies(full_data['category'], prefix='category')

# Combine features
feature_matrix = pd.concat([full_data[['user_id', 'book_id', 'rating']], category_encoded], axis=1)

# Create user-book rating matrix
user_book_matrix = feature_matrix.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)

# Cache for faster distance calculation
book_vectors = user_book_matrix.T.to_numpy()
book_id_list = user_book_matrix.columns.to_list()

def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def recommend_books(user_id, book_title, n_recommendations=5):
    if book_title not in books['title'].values:
        return "Book not found in the dataset."

    if user_id not in users['user_id'].values:
        return "User ID not found."

    book_id = books[books['title'] == book_title]['book_id'].values[0]

    try:
        book_idx = book_id_list.index(book_id)
    except ValueError:
        return "Book ID not found in user-book matrix."

    user_age = users.loc[users['user_id'] == user_id, 'age'].values[0]
    input_vector = book_vectors[book_idx]

    distances = []
    for i, other_vector in enumerate(book_vectors):
        if i == book_idx:
            continue
        dist = euclidean_distance(input_vector, other_vector)
        distances.append((dist, book_id_list[i]))

    # Sort by similarity (smallest distance)
    distances.sort()
    recommendations = []
    for _, sim_book_id in distances:
        sim_book_info = books[books['book_id'] == sim_book_id]
        sim_title = sim_book_info['title'].values[0]
        sim_age = sim_book_info['target_age_group'].values[0]

        if user_age >= sim_age:
            recommendations.append(sim_title)

        if len(recommendations) >= n_recommendations:
            break

    return recommendations if recommendations else ["No suitable recommendations found."]

# --------------------------
# User Input
# --------------------------
try:
    user_id = int(input("Enter your User ID: "))
    book_title = input("Enter book title you like: ").strip()
    n_recommendations = int(input("Enter number of recommendations: "))

    recommended_books = recommend_books(user_id, book_title, n_recommendations)
    print(f"\nBooks similar to '{book_title}' for user ID {user_id}:\n")
    for i, rec in enumerate(recommended_books, 1):
        print(f"{i}. {rec}")

except ValueError:
    print("Invalid input. Please enter numeric values for User ID and recommendation count.")


***************BOOK RECOMMENDATION SYSTEM********************
**************************************************************
Enter your User ID: 15
Enter book title you like: Human
Enter number of recommendations: 5

Books similar to 'Human' for user ID 15:

1. Member stay serious
2. At material mother quality
3. Clear million entire feeling participant
4. Next single
5. There man management agency single
